In [1]:
import collections
import cv2
import geojson
import json
import matplotlib.pyplot as plt
import numpy as np
from osgeo import gdal
import pandas as pd
from PIL import Image
import os
from osgeo import gdal, osr
import shapely.geometry as shgeom

%matplotlib inline

In [2]:
from tanzania_challenge import utils

In [30]:
def pixel_to_coordinates(x, y, imfeatures):
    lat = int(imfeatures["west"] + (imfeatures["east"]-imfeatures["west"]) * x / imfeatures["width"])
    lon = int(imfeatures["south"] + (imfeatures["north"]-imfeatures["south"]) * y / imfeatures["height"])
    return lat, lon

In [32]:
def set_coordinates_as_x_y(lat, lon, srid):
    """Transform coordinates into a (x,y)-compatible projection

    Parameters
    ----------
    coordinates : dict of 4 float values
        Min-x, min-y, max-x and max-y coordinates with keys 'west', 'south',
    'east', 'north'
    image_filename : str
        Image path on the file system (will be used to get the original image
    projection)
    srid : int
        Geographical projection

    Returns
    -------
    dict
        Bounding box of the image (west, south, east, north coordinates)
    """
    source = osr.SpatialReference()
    source.ImportFromEPSG(srid)
    target = osr.SpatialReference()
    target.ImportFromEPSG(4326)
    transform = osr.CoordinateTransformation(source, target)
    x, y = transform.TransformPoint(lat, lon)[:2]
    return y, x


In [33]:
def pixel_to_latlon(x, y, imfeatures):
    coordinates = pixel_to_coordinates(x, y, imfeatures)
    return set_coordinates_as_x_y(coordinates[0], coordinates[1], imfeatures["srid"])

In [35]:
def build_geom(building, imfeatures=None, pixel=False, min_x=2500, min_y=2500):
    feature = []
    for point in building:
        if pixel:
            feature.append((int(min_x + point[0][0]), min_y + int(point[0][1])))
        else:
            feature.append(pixel_to_latlon(point[0][0], point[0][1], imfeatures))
    feature.append(feature[0])
    return feature

In [110]:
def add_polygon(polygon, class_id, score, results, geofeatures, min_x=0, min_y=0):
    """
    """
    feature = build_geom(polygon, imfeatures=geofeatures, pixel=False, min_x=min_x, min_y=min_y)
    geom = geojson.Polygon([feature])
    shape = shgeom.shape(geom)
    pixel_feature = build_geom(polygon, pixel=True, min_x=0, min_y=0)
    pixel_geom = geojson.Polygon([pixel_feature])
    pixel_shape = shgeom.shape(pixel_geom)
    predictions = np.zeros([3])
    predictions[class_id-1] = score
    return results.append([*predictions, shape.wkt, pixel_shape.wkt])

In [101]:
def extract_geometry(mask, structure):
    """
    """
    denoised = cv2.morphologyEx(mask, cv2.MORPH_OPEN, structure)
    grown = cv2.morphologyEx(denoised, cv2.MORPH_CLOSE, structure)
    _, contours, _ = cv2.findContours(grown, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    polygons = [cv2.approxPolyDP(c, epsilon=0.01*cv2.arcLength(c, closed=True), closed=True)
                for c in contours]
    return polygons

## Exploit a Mask-RCNN result

In [112]:
instance = "5ae36dd70b093000130afdbb_384_384_19200_43392"
dataset = "open_ai_tanzania"
filename = os.path.join(datapath, dataset, "preprocessed", "384", "testing", "predicted_labels",
                       instance + ".json")
print(filename)
feature_filename = os.path.join(datapath, dataset, "preprocessed", "384", "testing", "features",
                       instance + ".json")
print(feature_filename)

/home/rde/data/open_ai_tanzania/preprocessed/384/testing/predicted_labels/5ae36dd70b093000130afdbb_384_384_19200_43392.json
/home/rde/data/open_ai_tanzania/preprocessed/384/testing/features/5ae36dd70b093000130afdbb_384_384_19200_43392.json


In [113]:
with open(filename) as f:
    prediction = json.load(f)

In [114]:
with open(feature_filename) as ff:
    geofeatures = json.load(ff)

In [115]:
geofeatures

{'west': 545028.9349441528,
 'south': 9332211.043327332,
 'east': 545055.0661430359,
 'north': 9332237.174526215,
 'srid': 32737,
 'width': 384,
 'height': 384}

In [116]:
masks = np.array(prediction["masks"], dtype=np.uint8)
class_ids = np.array(prediction["class_ids"], dtype=np.int8)
scores = np.array(prediction["scores"], dtype=np.float32)

In [117]:
class_ids, scores

(array([1], dtype=int8), array([0.8567583], dtype=float32))

In [122]:
results = []
for mask, class_id, score in zip(masks, class_ids, scores):
    polygon = extract_geometry(mask, structure)
    add_polygon(polygon[0], class_id, score, results, geofeatures, min_x=8832, min_y=39168)

In [123]:
df = pd.DataFrame(results, columns=["conf_completed", "conf_unfinished", "conf_foundation", "coords_geo", "coords_pixel"])

In [124]:
df

,conf_completed,conf_unfinished,conf_foundation,coords_geo,coords_pixel
0,0.856758,0.0,0.0,POLYGON ((-6.041229522125059 39.40690568661429...,"POLYGON ((1 154, 0 304, 139 303, 141 159, 1 154))"


In [125]:
df["coords_geo"][0]

'POLYGON ((-6.041229522125059 39.40690568661429, -6.041139068108067 39.40689658270442, -6.041139000489088 39.40698694639347, -6.041229461266385 39.40698701394789, -6.041229522125059 39.40690568661429))'